# Analyze (F)MNIST with `torch`

Careful: do **not** hit 'Kernel' > 'Restart & Run All', since some of the cells below take a long time to execute if you are not running the code on a GPU, so we already executed them for you. Only run the first few cells that are not yet executed.

In this notebook we compare different types of neural network architectures on the MNIST and Fashion MNIST datasets, to see how the performance improves when using a more complicated architecture. Additionally, we compare the networks to a simple logistic regression classifier from `sklearn`, which should have approximately the same accuracy as a linear FFNN (= a FFNN with only one layer mapping from the input directly to the output and no hidden layers, i.e., that has the same number of trainable parameters as the logistic regression model).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# torch neural network stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# torchvision includes the (F)MNIST datasets
from torchvision import datasets, transforms
# skorch provides a wrapper for torch networks so we can use them like sklearn models
from skorch import NeuralNetClassifier
from skorch.callbacks import EpochScoring
# set random seeds to get (at least more or less) reproducable results
np.random.seed(28)
torch.manual_seed(28);

## Load and look at the data

In [ ]:
# you do not need to understand what these functions do in detail

def torch_to_X_y(dataset):
    # transform input tensor to numpy array
    X = dataset.data.numpy()
    # reshape (28 x 28) pixel images to vector
    X = X.reshape(X.shape[0], -1).astype('float32')
    # the ToTensor transform was not applied to the raw data, so we need to scale ourselves
    X /= X.max()
    # extract numpy array with targets
    y = dataset.targets.numpy()
    return X, y

def load_data(use_fashion=False):
    if use_fashion:
        data_train = datasets.FashionMNIST("../data", train=True, download=True, transform=transforms.ToTensor())
        data_test = datasets.FashionMNIST("../data", train=False, transform=transforms.ToTensor())
    else:
        data_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
        data_test = datasets.MNIST("../data", train=False, transform=transforms.ToTensor())
    # extract (n_samples x n_features) and (n_samples,) X and y numpy arrays from torch dataset
    X_train, y_train = torch_to_X_y(data_train)
    X_test, y_test = torch_to_X_y(data_test)
    return X_train, X_test, y_train, y_test
    
def plot_images(x):
    n = 10
    plt.figure(figsize=(20, 4))
    for i in range(1, n+1):
        # display original
        ax = plt.subplot(2, n, i)
        plt.imshow(x[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

In [ ]:
# load and display the data -> see how the images have the same format
# MNIST
X_train, X_test, y_train, y_test = load_data()
plot_images(X_train)
# Fashion MNIST
X_train_F, X_test_F, y_train_F, y_test_F = load_data(use_fashion=True)
plot_images(X_train_F)

## See how a `torch` network works

In [ ]:
# FFNN with hidden layers (like the one you saw in the book)
class MyNeuralNet(nn.Module):
    
    def __init__(self, n_in=784, n_hl1=512, n_hl2=256, n_out=10, verbose=False):
        # input size is 28x28 pixel images flattened into a 784-dimensional vector
        # output size is 10 classes
        # hidden layer sizes can be set however you want
        super(MyNeuralNet, self).__init__()
        self.verbose = verbose
        # initialize layers
        self.l1 = nn.Linear(n_in, n_hl1)
        self.l2 = nn.Linear(n_hl1, n_hl2)
        self.lout = nn.Linear(n_hl2, n_out)
        
    def forward(self, x):
        # apply layers in correct order
        if self.verbose: print("[MyNeuralNet]  input:", x.shape)
        h = F.relu(self.l1(x))              # 784 -> 512 [relu]
        if self.verbose: print("[MyNeuralNet] 1st hl:", h.shape)
        h = F.relu(self.l2(h))              # 512 -> 256 [relu]
        if self.verbose: print("[MyNeuralNet] 2nd hl:", h.shape)
        y = F.softmax(self.lout(h), dim=1)  # 256 -> 10  [softmax]
        if self.verbose: print("[MyNeuralNet] output:", y.shape)
        return y

In [ ]:
# initialize the network
ffnn = MyNeuralNet(verbose=True)
# get an input data batch and convert the numpy array 
# to a torch tensor to use it with the network directly
# (skorch later works with the numpy arrays)
x = torch.Tensor(X_train[:16])
print(x.shape)  # batch size x features

In [ ]:
# apply network to input, i.e., call forward() to generate the prediction
y = ffnn(x)  # same as: y = ffnn.forward(x)
print(y.shape)  # batch size x classes

In [ ]:
# check the image of the first training sample
plt.imshow(X_train[0].reshape(28, 28));

In [ ]:
# look at the network's output for this first data point
# -> since the network wasn't trained yet, the predicted probabilities for all 10 classes are ~0.1
# (notice the grad parameter, which indicates that the network kept track of the gradients,
# which are needed for later tuning the weights during training)
y[0]

In [ ]:
# wrap torch NN in skorch Classifier and initialize
net = NeuralNetClassifier(
    MyNeuralNet,  # usually the class itself, not an instantiated object
    batch_size=32,  # how many samples are used in each training iteration
    optimizer=torch.optim.Adadelta,  # the optimizer (i.e. "what type" of gradient descent)
    lr=1.,  # learning rate of the optimizer
    device="cuda" if torch.cuda.is_available() else "cpu",  # train the network on a GPU if available
    max_epochs=1,  # for how many epochs to train the network
    callbacks=[  # additional stuff that should happen after each epoch, e.g., learning rate scheduler
        ('tr_acc', EpochScoring(  # or in this case print the accuracy after every epoch
            'accuracy',
            lower_is_better=False,
            on_train=True,
            name='train_acc',
        )),
    ],
)

# use simple sklearn-like interface to train the network (for 1 epoch)
net.fit(X_train, y_train)

In [ ]:
# generate predictions for the same samples as above
# -> this gives class labels directly like sklearn
y = net.predict(X_train[:16])

In [ ]:
# check if the prediction (after training) is correct
print("true class:", y_train[0])
y[0]

In [ ]:
# we can also get the original probabilities (notice the higher value at the index of the true class)
y = net.predict_proba(X_train[:16])
y[0]

## Define NNs for the classification task

In the code below, we define 3 different neural network architectures: a linear FFNN, a FFNN with multiple hidden layers, and a CNN, which is an architecture particularly well suited for image classification tasks.

You will see that the more complex architectures use an additional operation between layers called `Dropout`. This is a regularization technique used for training neural networks, where a certain percentage of the values in the hidden layer representation of a data point are randomly set to zero. You can think of this as the network suffering from a temporary stroke, which forces the neurons learn redundant representations (i.e., such that one neuron can take over for another neuron that was knocked out), which improves generalization.

In [13]:
# linear FFNN (--> same number of parameters as LogReg model)
class LinNN(nn.Module):
    
    def __init__(self, n_in=784, n_out=10):
        super(LinNN, self).__init__()
        self.l = nn.Linear(n_in, n_out)
        
    def forward(self, x):
        y = F.softmax(self.l(x), dim=1)  # 784 -> 10 [softmax]
        return y
    
# FFNN with hidden layers  
class FFNN(nn.Module):
    
    def __init__(self, n_in=784, n_hl1=512, n_hl2=256, n_out=10, dropout=0.2):
        super(FFNN, self).__init__()
        # initialize layers
        self.dropout = nn.Dropout(dropout)
        self.l1 = nn.Linear(n_in, n_hl1)
        self.l2 = nn.Linear(n_hl1, n_hl2)
        self.lout = nn.Linear(n_hl2, n_out)
        
    def forward(self, x):
        # apply layers in correct order
        h = F.relu(self.l1(x))              # 784 -> 512 [relu]
        h = self.dropout(h)
        h = F.relu(self.l2(h))              # 512 -> 256 [relu]
        h = self.dropout(h)
        y = F.softmax(self.lout(h), dim=1)  # 256 -> 10  [softmax]
        return y
    
# Convolutional Neural Net    
# based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        # convolutional and pooling layers
        h = self.conv1(x)
        h = F.relu(h)
        h = self.conv2(h)
        h = F.relu(h)
        h = F.max_pool2d(h, 2)
        h = self.dropout1(h)
        # flatten the representation and apply FFNN part for the classification
        h = torch.flatten(h, 1)
        h = self.fc1(h)
        h = F.relu(h)
        h = self.dropout2(h)
        h = self.fc2(h)
        y = F.softmax(h, dim=1)
        return y

# skorch wrapper with fit/predict methods
def eval_net(net_module, X_train, y_train, X_test, y_test, max_epochs=1):
    print("###", net_module.__name__)
    net = NeuralNetClassifier(
        net_module,
        batch_size=32,
        optimizer=torch.optim.Adadelta,
        lr=1.,
        device="cuda" if torch.cuda.is_available() else "cpu",
        max_epochs=max_epochs,
        callbacks=[
            ('tr_acc', EpochScoring(
                'accuracy',
                lower_is_better=False,
                on_train=True,
                name='train_acc',
            )),
        ],
    )
    net.fit(X_train, y_train)
    # evaluate on test set
    y_pred = net.predict(X_test)
    print('Test accuracy:', accuracy_score(y_test, y_pred), "\n")
    return net

## Test on MNIST dataset

As you see below, the simple logistic regression classifier is already very good on this easy task, with a test accuracy of over 92.6%.

The linear FFNN has almost the same accuracy (91.0%) as the LogReg model (please note: the NNs were only trained for a single epoch!) and the multi-layer FFNN is already better than the LogReg model (96.3%), while the CNN beats them all (98.2%), which is expected since this architecture is designed for the image classification task.

In [14]:
# get regular MNIST dataset
X_train, X_test, y_train, y_test = load_data()
# compare sklearn LogReg classifier
print("### LogReg")
clf = LogisticRegression(class_weight='balanced', random_state=1, fit_intercept=True)
clf.fit(X_train, y_train)
print('Test accuracy:', clf.score(X_test, y_test), "\n")
# and our different NN architectures
for net_module in [LinNN, FFNN, CNN]:
    if net_module == CNN:
        # the CNN operates on the 28x28 pixel images directly
        net = eval_net(net_module, X_train.reshape(-1, 1, 28, 28), y_train, X_test.reshape(-1, 1, 28, 28), y_test)
    else:
        # the FFNNs get the flattened vectors
        net = eval_net(net_module, X_train, y_train, X_test, y_test)

### LogReg


/home/franzi/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test accuracy: 0.9263 

### LinNN
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.8914        0.4044       0.9048        0.3267  1.8230
Test accuracy: 0.9097 

### FFNN
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9203        0.2596       0.9587        0.1487  3.6306
Test accuracy: 0.9625 

### CNN


/home/franzi/miniconda3/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9251        0.2474       0.9772        0.0786  7.1728
Test accuracy: 0.9824 



## Test on FashionMNIST

On the more difficult FMNIST task, the LogReg model has a much lower accuracy of 84.4% compared to the 92.6% achieved on the original MNIST dataset. When trained for only a single epoch, both the linear and multi-layer FFNNs have a lower accuracy than the LogReg model (81.2 and 82.4% respectively) and only the CNN does a bit better (87.3%). 

In [15]:
X_train, X_test, y_train, y_test = load_data(True)
# regular sklearn LogReg classifier
print("### LogReg")
clf = LogisticRegression(class_weight='balanced', random_state=1, fit_intercept=True)
clf.fit(X_train, y_train)
print('Test accuracy:', clf.score(X_test, y_test), "\n")
# our different NN
for net_module in [LinNN, FFNN, CNN]:
    if net_module == CNN:
        net = eval_net(net_module, X_train.reshape(-1, 1, 28, 28), y_train, X_test.reshape(-1, 1, 28, 28), y_test)
    else:
        net = eval_net(net_module, X_train, y_train, X_test, y_test)

### LogReg


/home/franzi/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test accuracy: 0.8438 

### LinNN
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.8015        0.5792       0.8253        0.5040  1.9656
Test accuracy: 0.8117 

### FFNN
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.7827        0.5929       0.8400        0.4453  3.8960
Test accuracy: 0.8236 

### CNN
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.8000        0.5616       0.8818        0.3272  7.0208
Test accuracy: 0.8728 



However, when trained for more epochs, the performance of all models improves, with the accuracy of the linear FFNN now being very close to that of the LogReg model (84.4%), while the multi-layer FFNN is better (87.1%) and the CNN can now solve the task quite well with an accuracy of 91.7%.

(See how the training and validation loss decrease over time - observing how these metrics develop can help you judge whether you've set your learning rate correctly and for how many epochs you should train the network.)

In [16]:
# train with more epochs
for net_module in [LinNN, FFNN, CNN]:
    if net_module == CNN:
        net = eval_net(net_module, X_train.reshape(-1, 1, 28, 28), y_train, X_test.reshape(-1, 1, 28, 28), y_test, max_epochs=15)
    else:
        net = eval_net(net_module, X_train, y_train, X_test, y_test, max_epochs=15)

### LinNN
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.8002        0.5800       0.8263        0.5020  2.0403
      2       0.8389        0.4715       0.8372        0.4734  2.1028
      3       0.8458        0.4520       0.8409        0.4620  2.1199
      4       0.8498        0.4416       0.8427        0.4563  2.2367
      5       0.8519        0.4347       0.8448        0.4533  2.1635
      6       0.8540        0.4297       0.8461        0.4516  2.1888
      7       0.8553        0.4258       0.8465        0.4509  2.1393
      8       0.8570        0.4227       0.8465        0.4506  2.1642
      9       0.8578        0.4201       0.8468        0.4507  2.2154
     10       0.8590        0.4179       0.8466        0.4510  2.3116
     11       0.8602        0.4160       0.8477        0.4515  2.1901
     12       0.8608        0.4143       0.8474        0.4520  2.1735
     13   